In [11]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

In [12]:
from PLSx.dataloader.dataloader import DataLoader
from torch import device, cuda
from PLSx.utils.file_manager import get_root
root = get_root(file="./", retrace=0)
print(root)

/Users/hadinayebi/CodingProjects/plsx


In [13]:
data_loader = DataLoader()
data_loader.train_data = "clss"
device = device("cuda:0" if cuda.is_available() else "cpu")

In [14]:
data_loader.transform_data(device)

In [15]:
len(data_loader.train_data)

178

In [16]:
_data_loader = data_loader.get_train_batch(batch_size=3)

In [17]:
for item in _data_loader:
    print(item)
    break

tensor([[20.,  8.,  0.],
        [20.,  8.,  0.],
        [20.,  8.,  0.],
        ...,
        [20.,  8.,  0.],
        [20.,  8.,  0.],
        [20.,  8.,  0.]])


In [18]:
item.shape

torch.Size([1068, 3])

In [33]:
from torch.nn.functional import one_hot

In [34]:
# from PLSx.autoencoder.autoencoder import Autoencoder
# from PLSx.autoencoder.architecture import Architecture

In [30]:
# architecture_json_file = root / "PLSx" / "test_data" / "architecture_sample.json"
# architecture = Architecture()
# architecture.build(architecture_json_file)
# ae = Autoencoder(architecture=architecture)


In [31]:
# input_ndx, target_vals_ss, target_vals_cl, one_hot_input = ae.transform_input(item, device, input_keys="ASC")

In [36]:
item[:, 0]

tensor([20., 20., 20.,  ..., 20., 20., 20.])

In [42]:
from numpy import array
from torch.nn import Sequential
from torch.nn import LSTM


In [37]:
# define input sequence
sequence = one_hot(item[:, 0].long(), num_classes=21) * 1.0
sequence.shape

torch.Size([1068, 21])

In [38]:
# reshape input into [samples, timesteps, features]
features = sequence.shape[1]
n_in = sequence.shape[0]
sequence = sequence.reshape((1, n_in, features))

In [39]:
sequence

tensor([[[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.]]])

In [45]:
import torch
import torch.nn as nn

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class Encoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=32):
        super(Encoder, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
        self.rnn1 = nn.LSTM(
        input_size=n_features,
        hidden_size=self.hidden_dim,
        num_layers=1,
        batch_first=True
        )
        self.rnn2 = nn.LSTM(
        input_size=self.hidden_dim,
        hidden_size=embedding_dim,
        num_layers=1,
        batch_first=True
        )

    def forward(self, x):
        x = x.reshape((1, self.seq_len, self.n_features))
        x, (_, _) = self.rnn1(x)
        x, (hidden_n, _) = self.rnn2(x)
        return hidden_n.reshape((self.n_features, self.embedding_dim))


class Decoder(nn.Module):
    def __init__(self, seq_len, n_features, input_dim=32):
        super(Decoder, self).__init__()
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 2 * input_dim, n_features
        self.rnn1 = nn.LSTM(
        input_size=input_dim,
        hidden_size=input_dim,
        num_layers=1,
        batch_first=True
        )
        self.rnn2 = nn.LSTM(
        input_size=input_dim,
        hidden_size=self.hidden_dim,
        num_layers=1,
        batch_first=True
        )
        self.output_layer = nn.Linear(self.hidden_dim, n_features)

    def forward(self, x):
        x = x.repeat(self.seq_len, self.n_features)
        x = x.reshape((self.n_features, self.seq_len, self.input_dim))
        x, (hidden_n, cell_n) = self.rnn1(x)
        x, (hidden_n, cell_n) = self.rnn2(x)
        x = x.reshape((self.seq_len, self.hidden_dim))
        return self.output_layer(x)


class LSTMAE(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=32):
        super(LSTMAE, self).__init__()
        self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
        self.decoder = Decoder(seq_len, n_features, embedding_dim).to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

TypeError: __init__() got an unexpected keyword argument 'activation'